In [ ]:
from importlib import reload

import bilby
import corner
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import numpy as np

import thesis_utils.plotting
reload(thesis_utils.plotting)
from thesis_utils import colours as thesis_colours
from thesis_utils.gw import get_cbc_parameter_labels
from thesis_utils.gw.quaternions import quaternion_to_euler, quaternion_to_source_angles, euler_to_quaternions
from thesis_utils.plotting import (
    get_default_corner_kwargs,
    set_plotting,
    save_figure,
    get_default_figsize,
    get_corner_figsize,
    plot_multiple_bilby,
)


set_plotting()

In [ ]:
quaternion_result_file = "outdir/quaternions_2023_03_07_result.json"
default_result_file = "outdir/defaults_2023_03_07_result.json"

In [ ]:
quaternion_result = bilby.core.result.read_in_result(quaternion_result_file)
default_result = bilby.core.result.read_in_result(default_result_file)

In [ ]:
print("Quaternions")
print(f"Log evidence: {quaternion_result.log_evidence} +/- {quaternion_result.log_evidence_err}")
print(f"Likelihood evaluations: {quaternion_result.num_likelihood_evaluations}")

In [ ]:

print(f"Log evidence: {default_result.log_evidence} +/- {default_result.log_evidence_err}")
print(f"Likelihood evaluations: {default_result.num_likelihood_evaluations}")

In [ ]:
quaternion_post = quaternion_result.posterior
default_post = default_result.posterior

In [ ]:
corner_kwargs = get_default_corner_kwargs()
corner_kwargs.pop("fill_contours")
corner_kwargs["show_titles"] = False
corner_kwargs["no_fill_contours"] = True
corner_kwargs["plot_datapoints"] = False
corner_kwargs.pop("color")

parameters = [p for p in default_result.search_parameter_keys if p != "geocent_time"]
colours=[thesis_colours.teal, thesis_colours.yellow]
labels=["Default", "Quaternions"]

fig = plot_multiple_bilby(
    [default_result, quaternion_result,],
    parameters=parameters,
    labels=labels,
    colours=colours,
    corner_labels=get_cbc_parameter_labels(default_result.search_parameter_keys, units=True),
    fig=plt.figure(figsize=get_corner_figsize(len(parameters))),
    add_legend=False,
    labelpad=0.1,
    **corner_kwargs,
)

handles = []
for l, c in zip(labels, colours):
    handles.append(
        Line2D([0], [0], color=c, label=l)
    )

fig.legend(
    handles=handles,
    loc="center",
    fontsize=14,
    bbox_to_anchor=(0.75, 0.75)
)
# axes = fig.get_axes()
# ndim = int(np.sqrt(len(axes)))
# axes[ndim - 1].get_legend().remove()
save_figure(fig, "quaternion_posterior_comparison", "figures")

In [ ]:
psi, theta_jn, phase = quaternion_post["psi"], quaternion_post["theta_jn"], quaternion_post["phase"]
dphase = np.mod(phase + np.sign(np.cos(theta_jn)) * psi, 2 * np.pi)

In [ ]:
fig = corner.corner(np.array([psi,theta_jn, dphase]).T)

In [ ]:
q_recon = np.array(euler_to_quaternions(psi, theta_jn, phase)).T
fig = corner.corner(q_recon)

In [ ]:
q_samples = quaternion_post[["q_0", "q_1", "q_2", "q_3"]].to_numpy()

In [ ]:
q_norm = q_samples / np.sqrt(np.sum(q_samples ** 2, axis=1)[:, np.newaxis])

In [ ]:
kwargs = get_default_corner_kwargs()
kwargs["no_fill_contours"] = True
kwargs["plot_datapoints"] = False
kwargs.pop("fill_contours")
kwargs["quantiles"] = []
kwargs["show_titles"] = False
kwargs["color"] = thesis_colours.teal
kwargs["hist_kwargs"]["color"] = kwargs["color"]

fig = plt.figure(figsize=get_corner_figsize(q_samples.shape[1]))
fig = corner.corner(
    q_samples,
    labels=[fr"$q_{i}$" for i in range(4)],
    labelpad=0.0,
    fig=fig,
    **kwargs
)
save_figure(fig, "quaternion_posterior", "figures")

In [ ]:
fig = plt.figure(figsize=get_corner_figsize(q_norm.shape[1]))
fig = corner.corner(
    q_norm,
    labels=[fr"$q_{i}$" for i in range(4)],
    labelpad=-0.0,
    fig=fig,
    **kwargs
)
save_figure(fig, "quaternion_posterior_norm", "figures")

## Exploring the parameter

Making some plots to prep for my viva.

In [ ]:
from sklearn.cluster import KMeans
import seaborn as sns

In [ ]:
q_samples[:, [0, -1]]

In [ ]:
kmeans = KMeans(n_clusters=4).fit(q_samples[:, [0, -1]])

In [ ]:
euler_angles = quaternion_to_euler(q_samples)
fig = None
cluster_colours = sns.color_palette(n_colors=4)
for i, colour in enumerate(cluster_colours):
    # fig = None
    fig = corner.corner(
        euler_angles[kmeans.labels_ == i],# % 2 * np.pi,
        # labels=[fr"$q_{i}$" for i in range(4)],
        labelpad=-0.0,
        fig=fig,
        color=colour,
        # **kwargs
    )
# save_figure(fig, "quaternion_posterior_norm", "figures")

In [ ]:
source_angles = quaternion_to_source_angles(q_samples)
fig = None
cluster_colours = sns.color_palette(n_colors=4)
for i, colour in enumerate(cluster_colours):
    fig = corner.corner(
        source_angles[kmeans.labels_ == i],
        # labels=[fr"$q_{i}$" for i in range(4)],
        labelpad=-0.0,
        fig=fig,
        color=colour,
        # **kwargs
    )